In [ ]:
import torch
import matplotlib.pyplot as pyplot
import numpy as np
import pandas as pd
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from tqdm.auto import tqdm
import torchvision.models as models
import cv2

In [ ]:
# hyperparams

batch_size = 32 
device = "cuda" if torch.cuda.is_available() else "cpu"
epochs = 3
lr = 1e-5

In [ ]:
imagenet_normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.RandomAffine(degrees=30, translate=(0.1, 0.1)),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.ToTensor(),
    imagenet_normalize
])


test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    imagenet_normalize
])


In [ ]:
model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)

for param in model.features[-5:].parameters():
    param.requires_grad = True



model.classifier = nn.Sequential(
    nn.Linear(25088, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 1)  
)

model.to(device)

In [ ]:
def train_loop(epochs, loader, model, loss_fn, optimizer):
    total_batches = len(loader)

    for epoch in tqdm(range(epochs)):

        total_epoch_loss = 0
        print(f"\n -------- Epoch {epoch} --------")
        
        for batch, (X, Y) in enumerate(loader):
            model.train()

            X, Y = X.to(device), Y.to(device)

            Y_pred = model(X)
            Y = Y.unsqueeze(1).float()
            loss = loss_fn(Y_pred, Y)
            total_epoch_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


        total_epoch_loss /= total_batches
        print()
        print(f"Avg training loss: {total_epoch_loss:.2f}")


def test_loop(loader, model, loss_fn):
    
    total_batches = len(loader)

    model.eval()
    total_loss, total_acc = 0, 0
    with torch.inference_mode():

        for X, Y in loader:
            
            X, Y = X.to(device), Y.to(device)

            Y_pred = model(X)
            Y = Y.unsqueeze(1).float()
            loss = loss_fn(Y_pred, Y)
            total_loss += loss.item()

            probs = torch.sigmoid(Y_pred)
            preds = (probs > 0.5).float()
            correct = (preds == Y).sum().item()
            total_acc += correct

        total_loss /= total_batches
        total_acc = total_acc / len(loader.dataset)

    return total_acc
        


In [ ]:
train_dataset = datasets.ImageFolder(root=r"C:\Users\HP\Desktop\phone-detector\Train-6", transform=train_transforms)
test_dataset  = datasets.ImageFolder(root=r"C:\Users\HP\Desktop\phone-detector\Test-2",  transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [ ]:
from ray import tune
from ray.tune.search.optuna import OptunaSearch
from ray.tune import TuneConfig

def custom_trial_dir_name(trial):
    return f"objective_lr{trial.config['lr']:.4f}_d{trial.config['dropout']}_f{trial.config['features']}"


def objective(config): 
    model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1) 
    
    for param in model.features[config['features']:].parameters():
        param.requires_grad = True


    model.classifier = nn.Sequential(
        nn.Linear(25088, 512),
        nn.ReLU(),
        nn.Dropout(config['dropout']),
        nn.Linear(512, 1)  
    )

    model.to(device)

    optimizer = torch.optim.AdamW([
        {"params": model.features[config['features_to_train']:].parameters(), "lr": config['lr']},  
        {"params": model.classifier.parameters(), "lr": config['clr']},     
    ], weight_decay=config['weight_decay'])

    loss_fn = torch.nn.BCEWithLogitsLoss()


    train_loop(config['epochs'], train_loader, model, loss_fn, optimizer)
    acc = test_loop(test_loader, model, loss_fn)
    tune.report({"mean_accuracy": acc})  


search_space = {"lr": tune.loguniform(1e-5,  1e-3),  
                "dropout" : tune.choice([0.2, 0.3, 0.4, 0.5]),
                "clr" : tune.loguniform(1e-5,  1e-3),
                "weight_decay" : tune.loguniform(1e-5,  1e-3),
                "features" : tune.choice([-2, -3, -4, -5, -6, -7]),
                "features_to_train" : tune.choice([-2, -3, -4, -5, -6, -7]),
                "epochs" : tune.choice([2, 4, 6, 8, 10])
                }

algo = OptunaSearch() 

tuner = tune.Tuner(  
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_accuracy",
        mode="max",
        search_alg=algo,
        trial_dirname_creator=custom_trial_dir_name
    ),
    run_config=tune.RunConfig(
        stop={"training_iteration": 1},
    ),
    param_space=search_space,
)
results = tuner.fit()
print("Best config is:", results.get_best_result().config)